# Getting Started with the Agent Development Kit (ADK) with Neo4j

This notebook shows how to use the Vertex AI Agent Framework to setup a multi-agent system integrating Neo4j.

The Vertex AI ADK is a python-based SDK that empowers developers to build multi-agent applications with custom logic, tools, and integrations.

In this notebook you will learn how to do the following:

*  Install Agent Development Kit
*  Define functions as tools
*  Define a database and analyst agent
*  Create a "Bom & Supplier Research" Multi-Agent


## Learn more about Neo4j

* https://neo4j.com/genai
* https://neo4j.com/developer
* https://graphrag.com
* https://neo4j.com/labs/genai-ecosystem
* [Neo4j and MCP Toolbox](https://neo4j.com/blog/developer/ai-agents-gen-ai-toolbox/)

## Learn more about the ADK

* [Launch Blog](https://cloud.google.com/blog/products/ai-machine-learning/build-and-manage-multi-system-agents-with-vertex-ai)
* [Developer Blog](https://developers.googleblog.com/en/agent-development-kit-easy-to-build-multi-agent-applications/)
* https://pypi.org/project/google-adk/
* https://google.github.io/adk-docs/
* https://github.com/google/adk-samples/
* [LLM Agents](https://google.github.io/adk-docs/agents/llm-agents/#putting-it-together-example)
* [Deploy to Agent Engine](https://google.github.io/adk-docs/deploy/agent-engine/)

## Use Case Focus For Our Agent
- __Summarize BOM Trees & Dependencies for Product:__ Can you summarize the BOM and potential country dependencies for product `<sku_id>`?
- __Find Dependant Products:__ What products depend on `<country>` for `<material>`?
- __Supplier Substitution:__ Supplier `<code>` is out of operation. I need to source alternatives.  Can you help?


# Install the Agent Development Kit

In [ ]:
#%pip install --upgrade google-cloud-aiplatform[adk,agent_engines] neo4j-rust-ext python-dotenv

# Load Env Variables

In [17]:
# from google.colab import auth
# auth.authenticate_user()
!gcloud auth application-default login > /dev/null 2>&1

In [18]:
from dotenv import load_dotenv
import getpass
import os

load_dotenv('nb.env', override=True)

if not os.environ.get('NEO4J_URI'):
    os.environ['NEO4J_URI'] = getpass.getpass('NEO4J_URI:\n')
if not os.environ.get('NEO4J_USERNAME'):
    os.environ['NEO4J_USERNAME'] = getpass.getpass('NEO4J_USERNAME:\n')
if not os.environ.get('NEO4J_PASSWORD'):
    os.environ['NEO4J_PASSWORD'] = getpass.getpass('NEO4J_PASSWORD:\n')

NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')

# Developer API or Vertex AI

The ADK supports two APIs:

* Google Gemini API (AI Studio)
* Vertex AI Gemini API.

In [19]:
# Only run this block for ML Developer API. Use your own API key.
import os

GOOGLE_API_KEY = os.environ.get('GOOGLE_API_KEY')
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "0"


In [ ]:
# Only run this block for Vertex AI API Use your own project / location.
# import os
#
# GOOGLE_CLOUD_PROJECT = "neo4jeventdemos" #@param {type:"string"}
#
# os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "1"
# os.environ["GOOGLE_CLOUD_PROJECT"] = GOOGLE_CLOUD_PROJECT
# os.environ["GOOGLE_CLOUD_LOCATION"] = "us-central1"

In [20]:
import logging

logger = logging.getLogger('agent_neo4j_cypher')
logger.info("Initializing Database for tools")

import logging

logging.getLogger("neo4j").setLevel(logging.ERROR)
logging.getLogger("google_genai").setLevel(logging.ERROR)

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="google_genai.types")
warnings.filterwarnings("ignore", category=UserWarning, module="neo4j.notifications")


# Graph Database Agent with `mcp-neo4j-cypher`
__Agent to answer simple database questions Using `neo4j-cypher` Model Context Protocol (MCP) Server for natural language queries.__

An AI agent reasons, plans, and takes actions.

The agent takes actions via access to **tools**, deciding how and when to invoke a tool. The agent also manages orchestration, creating a plan for answering a user query and adapting to responses that aren't quite correct.

Agent **tools** can be Python functions or MCP servers, in this case it is MCP.

## Creating the Database Agent

* display name
* instructions - detailed, making it clear exactly how the agent should behave and which tools to use when
* tools

In [21]:
from google.adk.models.lite_llm import LiteLlm
from google.adk.agents import Agent
from google.adk.tools.mcp_tool.mcp_toolset import MCPToolset, StdioServerParameters

database_agent = Agent(
    name="graph_database_agent",
    # model="gemini-2.0-flash-exp",
    # model=LiteLlm(model="openai/gpt-4o"),
    model=LiteLlm(model="anthropic/claude-sonnet-4-20250514"),
    description="""
    Agent to access knowledge graph stored in graph database
    """,
    instruction="""
      You are an Neo4j graph database and Cypher query expert, that must use the database schema with a user question and repeatedly generate valid cypher statements
      to execute on the database and answer the user's questions in a friendly manner in natural language. You can also directly return a graph schema when requested.

      For generating queries:
      If in doubt the database schema is always prioritized when it comes to nodes-types (labels) or relationship-types or property names, never take the user's input at face value.
      If the user requests also render tables, charts or other artifacts with the query results.
      Always validate the correct node-labels at the end of a relationship based on the schema.

      If a query fails or doesn't return data, use the error response 3 times to try to fix the generated query and re-run it, don't return the error to the user.
      If you cannot fix the query, explain the issue to the user and apologize.

      Fetch the graph database schema first and keep it in session memory to access later for query generation.
      Keep results of previous executions in session memory and access if needed, for instance ids or other attributes of nodes to find them again
      removing the need to ask the user. This also allows for generating shorter, more focused and less error-prone queries
      to for drill downs, sequences and loops.
      If possible resolve names to primary keys or ids and use those for looking up entities.
      The schema always indicates *outgoing* relationship-types from an entity to another entity, the graph patterns read like english language.
      `company has supplier` would be the pattern `(o:Organization)-[:HAS_SUPPLIER]->(s:Organization)`

      To get the schema of a database use the `get_schema` tool without parameters. Store the response of the schema tool in session context
      to access later for query generation.

      To answer a user question generate one or more Cypher statements based on the database schema and the parts of the user question.
      If necessary resolve categorical attributes (like names, countries, industries, publications) first by retrieving them for a set of entities to translate from the user's request.
      Use the `read_neo4j_cypher` tool repeatedly with the Cypher statements, you MUST generate statements that use named query parameters with `$parameter` style names
      and MUST pass them as a second dictionary parameter to the tool, even if empty.
      Parameter data can come from the users requests, prior query results or additional lookup queries.
      After the data for the question has been sufficiently retrieved, pass the data and control back to the parent agent.
    """,
    tools=[MCPToolset(
        connection_params=StdioServerParameters(
            command='uvx',
            args=[
                "mcp-neo4j-cypher",
            ],
            env={ k: os.environ[k] for k in ["NEO4J_URI","NEO4J_USERNAME","NEO4J_PASSWORD"] }
        ),
        tool_filter=['get_neo4j_schema','read_neo4j_cypher']
    )]
)

In [24]:
from google.adk.runners import InMemoryRunner
from google.genai.types import Part, UserContent

APP_NAME = 'BOM & Supplier Database Agent'
USER_ID = 'Zach Blumenfeld'


runner = InMemoryRunner(app_name=APP_NAME, agent=database_agent)

session = await runner.session_service.create_session( app_name=runner.app_name, user_id=USER_ID)

async def run_prompt(new_message: str):
  content = UserContent(parts=[Part(text=new_message)])
  result = None
  async for event in runner.run_async(user_id=session.user_id, session_id=session.id, new_message=content):
    for part in event.content.parts:
      print(part.text, part.function_call, part.function_response)
      if part.text:
        result = part.text
  return result

In [25]:
from IPython.display import Markdown, display

res = await run_prompt('What countries does the supply chain depend on?')
print("\n\n\n\nFinal Response:")
display(Markdown(res))

I'll help you analyze the supply chain dependencies by country. First, let me get the database schema to understand the structure of the data. None None
None id='toolu_01Nm5ayDLa6uaQhcC6PiN15j' args={} name='get_neo4j_schema' None
None None will_continue=None scheduling=None id='toolu_01Nm5ayDLa6uaQhcC6PiN15j' name='get_neo4j_schema' response={'result': CallToolResult(meta=None, content=[TextContent(type='text', text='[{"label": "Supplier", "attributes": {"annual_spend": "FLOAT", "tier": "STRING", "sub_type": "STRING", "code": "STRING indexed"}, "relationships": {"LOCATED_AT": "GeoLocation"}}, {"label": "Item", "attributes": {"desc_embedding": "LIST", "description": "STRING", "family": "STRING", "is_finished_product": "BOOLEAN", "name": "STRING", "sku_id": "STRING indexed"}, "relationships": {"AT": "Supplier", "BOM": "Item", "DEPENDS_ON": "Country"}}, {"label": "Customer", "attributes": {"annual_revenue": "FLOAT", "code": "INTEGER indexed"}, "relationships": {"LOCATED_AT": "GeoLocation

Based on my analysis of the supply chain data, here's what the supply chain depends on in terms of countries:

## Major Supply Chain Dependencies by Country

The supply chain has significant dependencies across **38 countries worldwide**, with varying levels of criticality:

### **Top Tier Dependencies (High Impact)**

1. **United States** - 118 dependent items across 8 product families
   - Highest dependency with 40 suppliers across all tiers (Tier1-Tier6)
   - Critical for: Bolts, Tubing, Rubber Seals, Castings, Rods, Raw Wire, Precision Bolts, Fasteners

2. **China** - 89 dependent items across 8 product families  
   - Second highest with 33 suppliers across all tiers
   - Critical for: Fasteners, Tubing, Bolts, Castings, Rubber Seals, Raw Wire, Rods, Precision Bolts

3. **Australia** - 68 dependent items across 7 product families
   - 40 suppliers across all tiers
   - Critical for: Castings, Tubing, Fasteners, Raw Wire, Bolts, Rods, Precision Bolts

### **Significant Dependencies (Medium-High Impact)**

4. **Brazil** - 66 items (8 families, 40 suppliers)
5. **Germany** - 58 items (6 families, 40 suppliers) 
6. **India** - 44 items (8 families, 32 suppliers)
7. **Mexico** - 42 items (8 families, 33 suppliers)
8. **South Africa** - 41 items (7 families, 33 suppliers)

### **Notable Regional Dependencies**

- **Asia-Pacific**: Strong dependencies on China, Australia, India, South Korea, Taiwan, Japan, Indonesia, Vietnam, and Malaysia
- **Americas**: Major reliance on USA, Brazil, Mexico, Canada, Chile, Peru, and Colombia  
- **Europe**: Key dependencies on Germany, Poland, Sweden, Ukraine, Switzerland, Italy, Norway, Ireland, Iceland, and Belgium
- **Middle East/Africa**: Dependencies on South Africa, Guinea, Saudi Arabia, Qatar, Morocco, Egypt, and Zambia

### **Risk Profile**

The supply chain shows:
- **Geographic diversity** across 6 continents
- **Multi-tier supplier presence** in most major countries (Tier1 through Tier6)
- **Product family spread** - most major countries supply multiple product types
- **Concentration risk** - Top 3 countries (USA, China, Australia) account for 275 of 897 total dependent items (31%)

This analysis reveals a globally distributed but concentrated supply chain with significant dependencies on major industrial economies, particularly in manufacturing-intensive regions.

In [27]:
res = await run_prompt('What families of finished products are there?')
print("\n\n\n\nFinal Response:")
display(Markdown(res))

Based on my analysis of the database, here are the **11 families of finished products** available:

## Agricultural Equipment (Major Product Lines)

### **Core Farm Machinery** - 107,605 total products
1. **HayCollector** - 21,671 products
   - Equipment for collecting and gathering hay from fields

2. **FarmTractor** - 21,521 products  
   - Primary agricultural vehicles for pulling/powering other equipment

3. **FieldSprayer** - 21,519 products
   - Equipment for applying pesticides, herbicides, and fertilizers

4. **SeedPlanter** - 21,488 products
   - Machinery for planting seeds in prepared soil

5. **CropHarvester** - 21,406 products
   - Equipment for harvesting mature crops

## Subsystems & Components (62 total products)

### **Electronic/Control Systems**
6. **CabWiringUnit** - 17 products
   - Electrical wiring assemblies for equipment cabins

7. **ControlAssembly** - 9 products
   - Control systems and assemblies for equipment operation

### **Mechanical Systems**  
8. **Mac

Based on my analysis of the database, here are the **11 families of finished products** available:

## Agricultural Equipment (Major Product Lines)

### **Core Farm Machinery** - 107,605 total products
1. **HayCollector** - 21,671 products
   - Equipment for collecting and gathering hay from fields

2. **FarmTractor** - 21,521 products  
   - Primary agricultural vehicles for pulling/powering other equipment

3. **FieldSprayer** - 21,519 products
   - Equipment for applying pesticides, herbicides, and fertilizers

4. **SeedPlanter** - 21,488 products
   - Machinery for planting seeds in prepared soil

5. **CropHarvester** - 21,406 products
   - Equipment for harvesting mature crops

## Subsystems & Components (62 total products)

### **Electronic/Control Systems**
6. **CabWiringUnit** - 17 products
   - Electrical wiring assemblies for equipment cabins

7. **ControlAssembly** - 9 products
   - Control systems and assemblies for equipment operation

### **Mechanical Systems**  
8. **MachineRig** - 14 products
   - Structural frameworks and rigging systems

9. **DrivePlatform** - 10 products
   - Drive systems and platforms for equipment mobility

10. **ChassisFrame** - 9 products
    - Structural chassis and frame components

11. **HydraulicSystem** - 3 products
    - Hydraulic power and control systems

## Product Architecture

The finished products show a **hierarchical structure** where:
- **Large agricultural machines** (HayCollector, FarmTractor, etc.) are complex assemblies
- **Subsystem components** (CabWiringUnit, HydraulicSystem, etc.) are integrated into the larger machines
- Products have **multi-level Bill of Materials (BOM)** structures where finished products can contain other finished products as subassemblies

This represents a comprehensive agricultural equipment product line focused on **crop production machinery** with supporting **subsystems and components** for complete farming operations from planting through harvest.

# Defining Functions for an Expert Research Agent

We will define more expert tools to create an agent for answering more sophisticated questions.

1. **`get_finished_product_families`**
Retrieves a list of finished product families and the number of products in each family.
2. **`get_component_families`**
Lists all component families (non-finished products), including the count of components in each family.
3. **`get_item`**
Fetches properties of an item, identified by a unique SKU ID, which can represent a component or finished product.
4. **`get_known_item_country_dependencies`**
Retrieves all known country dependencies for a given item, covering sourcing & manufacturing, and supply chain relationships.
5. **`get_item_bill_of_materials`**
Gathers the hierarchical bill of materials (BOM) for a specific item.
6. **`get_standard_country_codes_and_names`**
Provides a standardized list of country codes and corresponding names for use within applications and reporting.
7. **`get_products_with_country_dependencies`**
Identifies products that have sourcing or manufacturing dependencies on specific countries.
8. **`get_supplier_substitutions`**
Suggests alternative suppliers when the specified supplier (identified by a code) is unavailable or out of operation.
9. **`get_finished_products_depending_on_items`**
Determines which finished products rely on specific items as components in their structure.


## Neo4j Database Connectivity

First we define a small class `neo4jDatabase` that manages connecting to Neo4j and running read queries for the expert tools.

In [28]:
from neo4j import GraphDatabase
from typing import Any, List, Dict
import re

class neo4jDatabase:
    def __init__(self,  neo4j_uri: str, neo4j_username: str, neo4j_password: str):
        """Initialize connection to the Neo4j database"""
        logger.debug(f"Initializing database connection to {neo4j_uri}")
        d = GraphDatabase.driver(neo4j_uri, auth=(neo4j_username, neo4j_password))
        d.verify_connectivity()
        self.driver = d

    def is_write_query(self, query: str) -> bool:
      return re.search(r"\b(MERGE|CREATE|SET|DELETE|REMOVE|ADD)\b", query, re.IGNORECASE) is not None

    #TODO: Use result transformer here to r: r.data()
    def _execute_query(self, query: str, params: Dict[str, Any] | None = None) -> List[Dict[str, Any]]:
        """Execute a Cypher query and return results as a list of dictionaries"""
        logger.debug(f"Executing query: {query}")
        try:
            if self.is_write_query(query):
                logger.error(f"Write query not supported {query}")
                raise "Write Queries are not supported in this agent"
            else:
                #TODO: Add Routing here for efficiency
                result = self.driver.execute_query(query, params)
                results = [dict(r) for r in result.records]
                logger.debug(f"Read query returned {len(results)} rows")
                return results
        except Exception as e:
            logger.error(f"Database error executing query: {e}\n{query}")
            raise

In [29]:
db = neo4jDatabase(NEO4J_URI, NEO4J_USERNAME, NEO4J_PASSWORD)

In [30]:
# Testing database connection
db._execute_query("RETURN 1")

[{'1': 1}]

Now we can define our functions

In [31]:
def get_finished_product_families() -> List[Dict[str, Any]]:
    """
    Get the different type of Product Families - for finished products only.
    Use this when asked for different types, families, or groups of finished products.
    """
    try:
        results = db._execute_query("""
        MATCH (i:Item)
        WHERE i.is_finished_product
        RETURN i.family AS family, count(*) AS numberOfProducts
        """)
        return results
    except Exception as e:
        return [{"error":str(e)}]
#test
get_finished_product_families()

[{'family': 'FarmTractor', 'numberOfProducts': 21521},
 {'family': 'SeedPlanter', 'numberOfProducts': 21488},
 {'family': 'FieldSprayer', 'numberOfProducts': 21519},
 {'family': 'CropHarvester', 'numberOfProducts': 21406},
 {'family': 'HayCollector', 'numberOfProducts': 21671},
 {'family': 'CabWiringUnit', 'numberOfProducts': 17},
 {'family': 'ControlAssembly', 'numberOfProducts': 9},
 {'family': 'MachineRig', 'numberOfProducts': 14},
 {'family': 'HydraulicSystem', 'numberOfProducts': 3},
 {'family': 'ChassisFrame', 'numberOfProducts': 9},
 {'family': 'DrivePlatform', 'numberOfProducts': 10}]

In [32]:
def get_component_families() -> List[Dict[str, Any]]:
    """
    Get the different type of Component Families - for components only -not finished products.
    Use this when asked for different types, families or groups of components and/or parts.
    """
    try:
        results = db._execute_query("""
        MATCH (i:Item)
        WHERE NOT i.is_finished_product
        RETURN i.family AS family, count(*) AS numberOfComponents
        """)
        return results
    except Exception as e:
        return [{"error":str(e)}]
#test
get_component_families()

[{'family': 'MachineRig', 'numberOfComponents': 20017},
 {'family': 'DrivePlatform', 'numberOfComponents': 19896},
 {'family': 'HydraulicSystem', 'numberOfComponents': 20192},
 {'family': 'ChassisFrame', 'numberOfComponents': 19893},
 {'family': 'ControlAssembly', 'numberOfComponents': 19824},
 {'family': 'CabWiringUnit', 'numberOfComponents': 19886},
 {'family': 'Bushing', 'numberOfComponents': 162},
 {'family': 'Spring', 'numberOfComponents': 144},
 {'family': 'GearSet', 'numberOfComponents': 169},
 {'family': 'ElectricalControlBox', 'numberOfComponents': 77},
 {'family': 'ComponentModule', 'numberOfComponents': 63},
 {'family': 'Board', 'numberOfComponents': 82},
 {'family': 'SteeringColumn', 'numberOfComponents': 65},
 {'family': 'FrameSegment', 'numberOfComponents': 62},
 {'family': 'SensorModule', 'numberOfComponents': 67},
 {'family': 'OperatorCab', 'numberOfComponents': 12},
 {'family': 'AxleAssembly', 'numberOfComponents': 79},
 {'family': 'ControlUnit', 'numberOfComponents': 

In [33]:
def get_item(sku_id: str) -> dict[str, Any]:
    """
    Gets Item properties
    Args:
        sku_id (str): code uniquely identifying the item which can be a component or finished product
    Returns:
        dict[str, Any]: item properties
    """
    try:
        results = db._execute_query("""
        MATCH (i:Item {sku_id: $sku_id})
        RETURN i.sku_id AS sku_id, i.name AS name, i.family AS family, i.is_finished_product AS is_finished_product
        """, {"sku_id": sku_id})
        return results
    except Exception as e:
        return {"error": str(e)}

In [34]:
get_item("35472245A")

[{'sku_id': '35472245A',
  'name': 'FarmTractor_7YUC0',
  'family': 'FarmTractor',
  'is_finished_product': True}]

In [35]:
def get_known_item_country_dependencies(sku_id: str) -> List[Dict[str, Any]]:
    """
    Retrieve all known country dependencies for a given item, which can represent either a finished product, a component, or both. This includes data on sourcing, manufacturing, or other relationships tied to specific countries.
    Dependencies are determined recursively by examining all connected components in the item's bill of materials (BOM). For each component, this function identifies:
    1. **Known Country Dependencies:** A list of countries and their corresponding dependency descriptions (e.g., sourcing steel, manufacturing).
    2. **Supply Chain Dependencies:** A visual representation of how components with such country dependencies are linked to the provided item through potentially multiple stages of the BOM hierarchy, represented as a sequence of components connected by arrows, culminating in the provided item.

    Note: This function only retrieves available data, which may be incomplete due to limited internal intelligence.

    Args:
        sku_id (str): code uniquely identifying the component or finished product
    Returns:
        list[dict[str, Any]]: a list of components in this items bill of materials with country dependencies along with their country dependencies and how they relate to the item in the bill of materials hierarchy.
    """
    try:
        results = db._execute_query("""
        MATCH (i:Item {sku_id: $sku_id})
        MATCH path=(i)<-[:BOM*]-(comp:Item)-[r:DEPENDS_ON]->(c:Country)
        RETURN
        comp.sku_id AS sku_id,
        comp.name AS name,
        collect({country: c.name, dependency: r.description}) AS known_country_dependencies,
        ' <- ' + apoc.text.join([n IN tail(nodes(path))[..-1] | n.sku_id + ' (' + coalesce(n.name, '') + ')' ], ' <- ') AS supply_chain_dependency
        """, {"sku_id": sku_id})
        return results
    except Exception as e:
        return [{"error": str(e)}]

In [36]:
get_known_item_country_dependencies("35472245A")

[{'sku_id': 'M7307U91X',
  'name': 'RawWire_YCBEO',
  'known_country_dependencies': [{'dependency': 'sourcing raw copper cathode',
    'country': 'Chile'},
   {'dependency': 'drawing and annealing facilities', 'country': 'Mexico'},
   {'dependency': 'drawing and annealing facilities', 'country': 'Viet Nam'},
   {'dependency': 'sourcing raw copper cathode', 'country': 'Peru'}],
  'supply_chain_dependency': ' <- 35472245 (CabWiringUnit_Z5OO3) <- 35523080X (CabWiringUnit_UJ3OE) <- M7307U91X (RawWire_YCBEO)'},
 {'sku_id': 'M7307U92X',
  'name': 'PrecisionBolt_O12J1',
  'known_country_dependencies': [{'dependency': 'final machining and threading processes',
    'country': 'United States of America'},
   {'dependency': 'sourcing raw titanium sponge', 'country': 'Japan'},
   {'dependency': 'procuring alloying elements (aluminum and vanadium)',
    'country': 'Russian Federation'}],
  'supply_chain_dependency': ' <- 35472245 (CabWiringUnit_Z5OO3) <- 35523087X (CabWiringUnit_CZ0N2) <- M7307U92X

In [37]:
def get_item_bill_of_materials(sku_id: str) -> List[Dict[str, Any]]:
    """
    Retrieve the complete Bill of Materials (BOM) hierarchy for a given finished product or component identified by the SKU.
    This function provides a detailed view of all components involved in the production of the specified item.
    It identifies components recursively linked to the given item through the BOM hierarchy.

    Key Features:
    1. **Bill of Material Dependencies:** Identifies all components recursively linked to the given item through the BOM hierarchy.
    2. **Dependency Path Representation:** Produces a flattened tree structure where each entry represents a sequence of components
       connected by arrows, showing the hierarchical relationships from the base components to the provided item.


    Args:
        sku_id (str): The SKU (Stock Keeping Unit) that uniquely identifies a product, component, or part.

    Returns:
        list[dict[str, Any]]: A list of dependency paths represented as strings. Each path shows the components in the BOM hierarchy
        leading to the specified item, connected by arrows (` <- `). For example:
            " <- 35472245 (CabWiringUnit_Z5OO3)"
            " <- 35472245 (CabWiringUnit_Z5OO3) <- 35475154X (CabWiringUnit_VT75O)"
            " <- 35472245 (CabWiringUnit_Z5OO3) <- 35475154X (CabWiringUnit_VT75O) <- M3571002BX (CabWiringUnit_A9FH4)"
    """

    try:
        results = db._execute_query("""
        MATCH (i:Item {sku_id: $sku_id})
        MATCH path=(i)<-[:BOM*]-(:Item)
        RETURN ' <- ' + apoc.text.join([n IN tail(nodes(path)) | n.sku_id + ' (' + coalesce(n.name, '') + ')' ], ' <- ') AS supply_chain_dependency
        """, {"sku_id": sku_id})
        return results
    except Exception as e:
        return [{"error": str(e)}]


In [38]:
get_item_bill_of_materials("35472245A")

[{'supply_chain_dependency': ' <- 35472245 (CabWiringUnit_Z5OO3)'},
 {'supply_chain_dependency': ' <- 35472245 (CabWiringUnit_Z5OO3) <- M6495003X (DrivePlatform_SBK8U)'},
 {'supply_chain_dependency': ' <- 35472245 (CabWiringUnit_Z5OO3) <- M6499003X (MachineRig_RFYOE)'},
 {'supply_chain_dependency': ' <- 35472245 (CabWiringUnit_Z5OO3) <- M6499002X (HydraulicSystem_EE22P)'},
 {'supply_chain_dependency': ' <- 35472245 (CabWiringUnit_Z5OO3) <- 35370826 (MachineRig_INWWQ)'},
 {'supply_chain_dependency': ' <- 35472245 (CabWiringUnit_Z5OO3) <- 35433612 (ChassisFrame_2GN2W)'},
 {'supply_chain_dependency': ' <- 35472245 (CabWiringUnit_Z5OO3) <- 35475030 (ElectricalControlBox_6ROWJ)'},
 {'supply_chain_dependency': ' <- 35472245 (CabWiringUnit_Z5OO3) <- 28028534X (ControlAssembly_JRO9M)'},
 {'supply_chain_dependency': ' <- 35472245 (CabWiringUnit_Z5OO3) <- 35475154X (CabWiringUnit_VT75O)'},
 {'supply_chain_dependency': ' <- 35472245 (CabWiringUnit_Z5OO3) <- 35475154X (CabWiringUnit_VT75O) <- M357

In [39]:
def get_standard_country_codes_and_names() -> List[Dict[str, Any]]:
    """
    Get the standardized country codes and names that are mentioned in the BOM and supply chain data.
    Other methods that have country names and code as input will only match data if with these standardized codes/names.
    Note that not all countries may be listed, as not all countries are mentioned in intel.
    """
    try:
        results = db._execute_query("""
        MATCH (c:Country)
        RETURN c.code as code, c.name as name
        """)
        return results
    except Exception as e:
        return [{"error":str(e)}]
#test
get_standard_country_codes_and_names()

[{'code': 'AUS', 'name': 'Australia'},
 {'code': 'BEL', 'name': 'Belgium'},
 {'code': 'BRA', 'name': 'Brazil'},
 {'code': 'CAN', 'name': 'Canada'},
 {'code': 'CHE', 'name': 'Switzerland'},
 {'code': 'CHL', 'name': 'Chile'},
 {'code': 'CHN', 'name': 'China'},
 {'code': 'COL', 'name': 'Colombia'},
 {'code': 'DEU', 'name': 'Germany'},
 {'code': 'EGY', 'name': 'Egypt'},
 {'code': 'GIN', 'name': 'Guinea'},
 {'code': 'IDN', 'name': 'Indonesia'},
 {'code': 'IND', 'name': 'India'},
 {'code': 'IRL', 'name': 'Ireland'},
 {'code': 'ISL', 'name': 'Iceland'},
 {'code': 'ITA', 'name': 'Italy'},
 {'code': 'JPN', 'name': 'Japan'},
 {'code': 'KAZ', 'name': 'Kazakhstan'},
 {'code': 'KOR', 'name': 'Korea, Republic of'},
 {'code': 'MAR', 'name': 'Morocco'},
 {'code': 'MEX', 'name': 'Mexico'},
 {'code': 'MYS', 'name': 'Malaysia'},
 {'code': 'NOR', 'name': 'Norway'},
 {'code': 'PER', 'name': 'Peru'},
 {'code': 'PHL', 'name': 'Philippines'},
 {'code': 'POL', 'name': 'Poland'},
 {'code': 'QAT', 'name': 'Qatar

In [42]:
from typing import List, Dict, Optional


def get_products_with_country_dependencies(country_code:str, dependency_search_term: Optional[str] = None) -> List[Dict[str, Any]]:
    """
    Identify finished products that are affected by dependencies on a specific material or resource from a given country.

    This function traces the supply chain of finished products to specific materials or resources sourced from a particular country. By analyzing the Bill of Materials (BOM) hierarchy,
    it identifies how dependencies on components are linked to the production of finished products.

    Key Features:
    1. **Country-Specific Dependencies:** Finds components linked to a specific country and determines how their dependencies
       impact finished products.
    2. **Dependency Search:** Allows optional filtering of dependencies by a search term (e.g., "nickel").
    3. **Supply Chain Traceability:** Provides detailed paths showing how the affected components feed into finished products.

    Args:
        country_code (str): The standardized ISO 3166 alpha-3 country code (e.g., "CAN" for Canada) to trace dependencies related to a specific country.
        dependency_search_term (Optional[str]): A search term to filter specific dependencies (e.g., "nickel").
            If omitted, all dependencies for the country are considered. keep the search terms short and simple - exact matching is used for criteria

    Returns:
        List[Dict[str, Any]]: A list where each element represents a finished product impacted by the specified dependencies.
        Each entry includes:
            - **product_sku_id (str):** The SKU of the finished product.
            - **product_name (str):** The name of the finished product.
            - **dependencies (list):** A list of dictionaries, with each dictionary representing:
                - **depends_on_item_sku_id (str):** The SKU of the component causing the dependency.
                - **depends_on_item_name (str):** The name of the component causing the dependency.
                - **country_dependency_of_item (list[dict]):** A list of country dependencies associated with the component,
                  including:
                    - **dependency (str):** A description of the dependency (e.g., "sourcing nickel").
                    - **country (str):** The country associated with the dependency (e.g., "Canada").
                - **supply_chain_dependency_for_product (str):** A trace of the supply chain relationship between the component
                  and the finished product. This trace is represented as a sequence of SKUs and component names connected by arrows (e.g.,
                  " <- D0301002 (PrecisionBolt_252A1) <- 13952372 (ChassisFrame_3AC3Z)").

    Examples:
        - **Query:** country_code = "CAN", dependency_search_term = "nickel"
        - **Output:**
            [
                {
                    "product_sku_id": "35513559",
                    "product_name": "FarmTractor_VLM4F",
                    "dependencies": [
                        {
                            "depends_on_item_sku_id": "D0301002",
                            "depends_on_item_name": "PrecisionBolt_252A1",
                            "country_dependency_of_item": [
                                {
                                    "dependency": "sourcing nickel",
                                    "country": "Canada"
                                }
                            ],
                            "supply_chain_dependency_for_product": " <- D0301002 (PrecisionBolt_252A1) <- 13978746 (CabWiringUnit_EVKCK)"
                        }
                    ]
                },
                ...
            ]
    """

    try:
        if dependency_search_term is None:
            results = db._execute_query("""
            MATCH (i:Item)-[r:DEPENDS_ON]->(c:Country {code:$countryCode})
            WITH i,r,c
            MATCH path = (i)-[:BOM*]->(p {is_finished_product:true})
            WITH p.sku_id AS product_sku_id, p.name AS product_name,  i.sku_id AS depends_on_item_sku_id, i.name AS depends_on_item_name, collect({country: c.name, dependency: r.description}) AS known_country_dependency,
                ' <- ' + apoc.text.join([n IN reverse(nodes(path)[..-1]) | n.sku_id + ' (' + coalesce(n.name, '') + ')' ], ' <- ') AS supply_chain_dependency
            RETURN  product_sku_id, product_name, collect({depends_on_item_sku_id: depends_on_item_sku_id, depends_on_item_name: depends_on_item_name, country_dependency_of_item: known_country_dependency, supply_chain_dependency_for_product: supply_chain_dependency}) AS dependencies
            """, {"countryCode": country_code, "dependencySearchTerm": dependency_search_term})
        else:
            results = db._execute_query("""
            MATCH (i:Item)-[r:DEPENDS_ON]->(c:Country {code:$countryCode})
            WHERE  lower(r.description) CONTAINS lower($dependencySearchTerm)
            WITH i,r,c
            MATCH path = (i)-[:BOM*]->(p {is_finished_product:true})
            WITH p.sku_id AS product_sku_id, p.name AS product_name,  i.sku_id AS depends_on_item_sku_id, i.name AS depends_on_item_name, collect({country: c.name, dependency: r.description}) AS known_country_dependency,
                ' <- ' + apoc.text.join([n IN reverse(nodes(path)[..-1])| n.sku_id + ' (' + coalesce(n.name, '') + ')' ], ' <- ') AS supply_chain_dependency
            RETURN  product_sku_id, product_name, collect({depends_on_item_sku_id: depends_on_item_sku_id, depends_on_item_name: depends_on_item_name, country_dependency_of_item: known_country_dependency, supply_chain_dependency_for_product: supply_chain_dependency}) AS dependencies
            """, {"countryCode": country_code, "dependencySearchTerm": dependency_search_term})
        return results
    except Exception as e:
        return [{"error": str(e)}]

In [43]:
get_products_with_country_dependencies("CAN", "nickel")[:2]

[{'product_sku_id': '35298723',
  'product_name': 'FieldSprayer_BE9WW',
  'dependencies': [{'supply_chain_dependency_for_product': ' <- E0301006 (PrecisionBolt_66KS7)',
    'depends_on_item_sku_id': 'E0301006',
    'depends_on_item_name': 'PrecisionBolt_66KS7',
    'country_dependency_of_item': [{'dependency': 'sourcing nickel for alloy',
      'country': 'Canada'}]}]},
 {'product_sku_id': '35298721',
  'product_name': 'SeedPlanter_DJG7M',
  'dependencies': [{'supply_chain_dependency_for_product': ' <- E0301006 (PrecisionBolt_66KS7)',
    'depends_on_item_sku_id': 'E0301006',
    'depends_on_item_name': 'PrecisionBolt_66KS7',
    'country_dependency_of_item': [{'dependency': 'sourcing nickel for alloy',
      'country': 'Canada'}]}]}]

In [44]:
get_products_with_country_dependencies("USA")[:2]

[{'product_sku_id': '35296458',
  'product_name': 'HayCollector_GSZGQ',
  'dependencies': [{'supply_chain_dependency_for_product': ' <- KM100605 (Bolt_HZU3P)',
    'depends_on_item_sku_id': 'KM100605',
    'depends_on_item_name': 'Bolt_HZU3P',
    'country_dependency_of_item': [{'dependency': 'finishing and coating',
      'country': 'United States of America'}]}]},
 {'product_sku_id': '35296455',
  'product_name': 'CropHarvester_5EWBJ',
  'dependencies': [{'supply_chain_dependency_for_product': ' <- KM100605 (Bolt_HZU3P)',
    'depends_on_item_sku_id': 'KM100605',
    'depends_on_item_name': 'Bolt_HZU3P',
    'country_dependency_of_item': [{'dependency': 'finishing and coating',
      'country': 'United States of America'}]}]}]

In [45]:
def get_supplier_substitutions(supplier_code: str) -> List[Dict[str, Any]]:
    """
    Retrieves substitution recommendations for items (components and/or finished products) supplied by a specific supplier.

    This function is designed to assist in scenarios where a supplier (identified by its unique code)
    is temporarily or permanently unable to operate or is otherwise designated for substitution.
    It identifies items supplied by the specified supplier and recommends other suppliers on the same BOM tier that
    can substitute those items. Recommendations are ranked by supplier proximity based on geodistance.

    Args:
        supplier_code (str): A unique code identifying the supplier in question.

    Returns:
        list[dict[str, Any]]: A list of dictionaries, each containing details for an item supplied by the specified supplier:
            - sku_id (str): The unique identifier for the item.
            - name (str): The name or description of the item.
            - recommended_supplier (dict): The closest alternative supplier for the item, including:
                - code (str): The unique code for the supplier.
                - distance (float): The geodistance (in miles) between the original supplier's location and the alternate supplier.
              This is `null` if no alternative suppliers are available.
            - other_suppliers (list[dict]): A list of additional alternative suppliers, sorted by ascending distance, with each supplier having:
                - code (str): The unique code for the supplier.
                - distance (float): The geodistance (in miles).
              This is an empty list if no additional alternatives are available.

    Example Output:
        [
            {
                "sku_id": "28710197",
                "name": "Harness_YU5KA",
                "recommended_supplier": {
                    "distance": 99.65,
                    "code": "T4J5DE"
                },
                "other_suppliers": []
            },
            {
                "sku_id": "28018274",
                "name": "Clamp_MRVR0",
                "recommended_supplier": {
                    "distance": 99.65,
                    "code": "T4J5DE"
                },
                "other_suppliers": [
                    {
                        "distance": 1376.98,
                        "code": "RO6UY3"
                    },
                    {
                        "distance": 11170.08,
                        "code": "6TIPXK"
                    }
                ]
            }
        ]

    """
    try:
        results = db._execute_query("""
        MATCH (l)<-[:LOCATED_AT]-(s:Supplier {code: $code})<-[:AT]-(i:Item)
        OPTIONAL MATCH (i)-[:AT]->(sAlt:Supplier  WHERE sAlt.tier = s.tier AND sAlt <> s)-[:LOCATED_AT]->(lAlt)
        WHERE sAlt.tier = s.tier AND sAlt <> s
        // Calculate geodistance
        WITH i, sAlt, round(point.distance(l.geo_point, lAlt.geo_point) * 0.000621371) AS distance_in_miles
        ORDER BY i, distance_in_miles ASC
        // Collect recommended supplier and other options
        WITH i, collect({code: sAlt.code, distance: distance_in_miles}) AS all_sAlt
        RETURN i.sku_id AS sku_id, i.name as name, all_sAlt[0] AS recommended_supplier, tail(all_sAlt) AS other_suppliers
        """, {"code":supplier_code})
        return results
    except Exception as e:
        return [{"error":str(e)}]

In [46]:
def get_finished_products_depending_on_items(sku_ids: List[str]) -> List[Dict[str, Any]]:
    """
    Find finished products that depend on the provided item SKUs in their supply chain.

    This function identifies finished products in a supply chain that directly or indirectly rely
    on specific items (provided by their SKUs). Using the Bill of Materials (BOM) relationships,
    it traces the supply chain dependencies starting from the supplied `sku_ids` and identifies
    finished products (`is_finished_product == true`). It also generates a detailed dependency path
    for each finished product, showing how the input SKUs are linked to the finished product.

    Args:
        sku_ids (List[str]): A list of item SKUs to find dependencies for.

    Returns:
        List[Dict[str, Any]]: A list of dictionaries where each entry corresponds to a finished product.
            Each dictionary contains:
            - `sku_id` (str): The SKU of the finished product.
            - `depends_on_item_sku_ids` (List[str]): A list of unique item SKUs from the input that this
              finished product depends on.
            - `supply_chain_dependency` (List[str]): A list of dependency paths starting from the finished
              product and tracing backward to the input SKUs. Each path is represented as a string,
              detailing the SKUs and names of items connected in the workflow. The format of each
              path is:
              ```
              "<- FinishedProductSKU (FinishedProductName) <- IntermediateSKU (IntermediateName) <- InputSKU (InputName)"
              ```
              If there are multiple paths for the same input SKU leading to the finished product, all
              paths are included.

    Example Output:
        [
            {
                "sku_id": "13867580",
                "depends_on_item_sku_ids": ["B0301002"],
                "supply_chain_dependency": [
                    " <- 13578113 (DrivePlatform_B6A4T) <- B0301002 (Bolt_W5F3Y)"
                ]
            },
            {
                "sku_id": "35750243",
                "depends_on_item_sku_ids": ["B0301002"],
                "supply_chain_dependency": [
                    " <- 13578113 (DrivePlatform_B6A4T) <- B0301002 (Bolt_W5F3Y)",
                    " <- 13599782 (MachineRig_IBA4I) <- B0301002 (Bolt_W5F3Y)"
                ]
            }
        ]

    """
    try:
        results = db._execute_query("""
        MATCH path = (i:Item)-[:BOM*]->(p {is_finished_product: true})
        WHERE i.sku_id IN $skuIds

        RETURN p.sku_id AS sku_id,
          collect(DISTINCT i.sku_id) AS depends_on_item_sku_ids,
          collect(' <- ' + apoc.text.join([n IN reverse(nodes(path)[..-1]) | n.sku_id + ' (' + coalesce(n.name, '') + ')' ], ' <- ')) AS supply_chain_dependency
        """, {"skuIds":sku_ids})
        return results
    except Exception as e:
        return [{"error":str(e)}]

In [47]:
get_finished_products_depending_on_items(["KM100347", "B0301002", "KM100322"])[:3]

[{'sku_id': '13867580',
  'depends_on_item_sku_ids': ['B0301002'],
  'supply_chain_dependency': [' <- 13578113 (DrivePlatform_B6A4T) <- B0301002 (Bolt_W5F3Y)']},
 {'sku_id': '35750245',
  'depends_on_item_sku_ids': ['B0301002'],
  'supply_chain_dependency': [' <- 13578113 (DrivePlatform_B6A4T) <- B0301002 (Bolt_W5F3Y)']},
 {'sku_id': '35750243',
  'depends_on_item_sku_ids': ['B0301002'],
  'supply_chain_dependency': [' <- 13578113 (DrivePlatform_B6A4T) <- B0301002 (Bolt_W5F3Y)',
   ' <- 13599782 (MachineRig_IBA4I) <- B0301002 (Bolt_W5F3Y)']}]

## Define Expert Agent




In [48]:
from google.adk.agents import Agent

In [49]:
MODEL="gemini-2.5-pro-preview-03-25"

In [50]:
bom_supplier_research_agent = Agent(
    model=MODEL,
    name='bom_supplier_research_agent',
    instruction="""
    You are an agent that has access to a database of bill of materials (BOM), supplier, product/component, and customer relationships.
    Use the provided tools to answer questions. 
    when returning information, try to always return not just the factual attribute data but also
    codes, skus, and ids to allow the other agents to investigate them more.
    """,
    tools=[
        get_item,
        get_known_item_country_dependencies,
        get_item_bill_of_materials,
        get_standard_country_codes_and_names,
        get_finished_product_families,
        get_products_with_country_dependencies,
        get_component_families,
        get_supplier_substitutions,
        get_finished_products_depending_on_items
    ]
)

# Define Root Agent

In [51]:
root_agent = Agent(
    model=MODEL,
    name='bom_supplier_agent',
    global_instruction = "",
    instruction="""
    You are an agent that has access to a database of bill of materials (BOM), supplier, product/component, and customer relationships.
    You have a set of agents to retrieve information, you should prefer the research agents over the database agent - particularly for questions around bill of materials, component & product info, and country dependencies. Only use the database agent if you have to as a fallback.
    If the user requests it, do render tables, charts or other artifacts with the research results.
    """,

    sub_agents=[bom_supplier_research_agent, database_agent]
)

# Let's try it

You can [run the ADK locally](https://google.github.io/adk-docs/get-started/local-testing/#expected-output) a web application with `adk web` or the FastAPI server with `adk api_server`.

You can also deploy the agent to [Cloud Run](https://google.github.io/adk-docs/deploy/cloud-run/) or to [Agent Engine](https://google.github.io/adk-docs/deploy/agent-engine/).

In [52]:
APP_NAME = 'BOM & Supplier Analyst'
USER_ID = 'Zach Blumenfeld'

from google.adk.runners import InMemoryRunner
from google.genai.types import Part, UserContent


runner = InMemoryRunner(app_name=APP_NAME, agent=root_agent)

session = await runner.session_service.create_session(app_name=runner.app_name, user_id=USER_ID)

In [54]:
async def run_prompt(new_message: str):
  content = UserContent(parts=[Part(text=new_message)])
# print (content)
  result = None
  async for event in runner.run_async(user_id=USER_ID, session_id=session.id, new_message=content):
#    print(event.content.model_dump(exclude_none=True))
#    print(event.content.parts)
    for part in event.content.parts:
      print(part.text, part.function_call, part.function_response)
      if part.text:
#        print(part.text)
        result = part.text
  return result

In [55]:
from IPython.display import Markdown, display

res = await run_prompt('what types of finished products are there?')
print("\n\n\n\nFinal Response:")
display(Markdown(res))

None id='adk-09c31298-d001-499f-aee2-6c6d73e942e0' args={'agent_name': 'bom_supplier_research_agent'} name='transfer_to_agent' None
None None will_continue=None scheduling=None id='adk-09c31298-d001-499f-aee2-6c6d73e942e0' name='transfer_to_agent' response={}
None id='adk-ca71be1e-1fc9-4362-903a-e6b32b43ce56' args={} name='get_finished_product_families' None
None None will_continue=None scheduling=None id='adk-ca71be1e-1fc9-4362-903a-e6b32b43ce56' name='get_finished_product_families' response={'result': [{'family': 'FarmTractor', 'numberOfProducts': 21521}, {'family': 'SeedPlanter', 'numberOfProducts': 21488}, {'family': 'FieldSprayer', 'numberOfProducts': 21519}, {'family': 'CropHarvester', 'numberOfProducts': 21406}, {'family': 'HayCollector', 'numberOfProducts': 21671}, {'family': 'CabWiringUnit', 'numberOfProducts': 17}, {'family': 'ControlAssembly', 'numberOfProducts': 9}, {'family': 'MachineRig', 'numberOfProducts': 14}, {'family': 'HydraulicSystem', 'numberOfProducts': 3}, {'fam

The following are the types of finished product families:
FarmTractor: 21521 products
SeedPlanter: 21488 products
FieldSprayer: 21519 products
CropHarvester: 21406 products
HayCollector: 21671 products
CabWiringUnit: 17 products
ControlAssembly: 9 products
MachineRig: 14 products
HydraulicSystem: 3 products
ChassisFrame: 9 products
DrivePlatform: 10 products

In [56]:
res = await run_prompt('Can you summarize the BOM and potential country dependencies for product "35472245A"')
print("\n\n\n\nFinal Response:")
display(Markdown(res))

None id='adk-ea35e515-ce29-44bd-aef7-207f6438736f' args={'sku_id': '35472245A'} name='get_item_bill_of_materials' None
None id='adk-b37901d8-1876-4dde-bd88-68fb9f7c492f' args={'sku_id': '35472245A'} name='get_known_item_country_dependencies' None
None None will_continue=None scheduling=None id='adk-ea35e515-ce29-44bd-aef7-207f6438736f' name='get_item_bill_of_materials' response={'result': [{'supply_chain_dependency': ' <- 35472245 (CabWiringUnit_Z5OO3)'}, {'supply_chain_dependency': ' <- 35472245 (CabWiringUnit_Z5OO3) <- M6495003X (DrivePlatform_SBK8U)'}, {'supply_chain_dependency': ' <- 35472245 (CabWiringUnit_Z5OO3) <- M6499003X (MachineRig_RFYOE)'}, {'supply_chain_dependency': ' <- 35472245 (CabWiringUnit_Z5OO3) <- M6499002X (HydraulicSystem_EE22P)'}, {'supply_chain_dependency': ' <- 35472245 (CabWiringUnit_Z5OO3) <- 35370826 (MachineRig_INWWQ)'}, {'supply_chain_dependency': ' <- 35472245 (CabWiringUnit_Z5OO3) <- 35433612 (ChassisFrame_2GN2W)'}, {'supply_chain_dependency': ' <- 3547

Summary for product 35472245A:

Bill of Materials:
The product 35472245A (CabWiringUnit_Z5OO3) has an extensive BOM. Here are a few examples of its dependencies:
*   Directly depends on: M6495003X (DrivePlatform_SBK8U), M6499003X (MachineRig_RFYOE), M6499002X (HydraulicSystem_EE22P), and many others.
*   Indirectly depends on items such as M3571002BX (CabWiringUnit_A9FH4) through 35475154X (CabWiringUnit_VT75O), and M7307U91X (RawWire_YCBEO) through 35523080X (CabWiringUnit_UJ3OE).

Potential Country Dependencies:
The product 35472245A has several potential country dependencies through its components:
*   RawWire_YCBEO (M7307U91X):
    *   Sourcing raw copper cathode: Chile, Peru
    *   Drawing and annealing facilities: Mexico, Viet Nam
    *   Supply chain path: ... <- 35523080X (CabWiringUnit_UJ3OE) <- M7307U91X (RawWire_YCBEO)
*   PrecisionBolt_O12J1 (M7307U92X):
    *   Final machining and threading processes: United States of America
    *   Sourcing raw titanium sponge: Japan
    *   Procuring alloying elements (aluminum and vanadium): Russian Federation
    *   Supply chain path: ... <- 35523087X (CabWiringUnit_CZ0N2) <- M7307U92X (PrecisionBolt_O12J1)
*   Fastener_CP82S (M7307U93X):
    *   Precision forging: Germany
    *   Sourcing raw titanium: Russian Federation
    *   Supply chain path: ... <- 35523089X (DrivePlatform_ZB6KE) <- M7307U93X (Fastener_CP82S)

This summary highlights the complexity of the supply chain for 35472245A and its potential vulnerabilities due to geographic dependencies.

In [57]:
res = await run_prompt('What products depend on Canada for nickel?')
print("\n\n\n\nFinal Response:")
display(Markdown(res))

None id='adk-9e2ca89a-71a6-45a4-a08c-e090dfff2c47' args={} name='get_standard_country_codes_and_names' None
None None will_continue=None scheduling=None id='adk-9e2ca89a-71a6-45a4-a08c-e090dfff2c47' name='get_standard_country_codes_and_names' response={'result': [{'code': 'AUS', 'name': 'Australia'}, {'code': 'BEL', 'name': 'Belgium'}, {'code': 'BRA', 'name': 'Brazil'}, {'code': 'CAN', 'name': 'Canada'}, {'code': 'CHE', 'name': 'Switzerland'}, {'code': 'CHL', 'name': 'Chile'}, {'code': 'CHN', 'name': 'China'}, {'code': 'COL', 'name': 'Colombia'}, {'code': 'DEU', 'name': 'Germany'}, {'code': 'EGY', 'name': 'Egypt'}, {'code': 'GIN', 'name': 'Guinea'}, {'code': 'IDN', 'name': 'Indonesia'}, {'code': 'IND', 'name': 'India'}, {'code': 'IRL', 'name': 'Ireland'}, {'code': 'ISL', 'name': 'Iceland'}, {'code': 'ITA', 'name': 'Italy'}, {'code': 'JPN', 'name': 'Japan'}, {'code': 'KAZ', 'name': 'Kazakhstan'}, {'code': 'KOR', 'name': 'Korea, Republic of'}, {'code': 'MAR', 'name': 'Morocco'}, {'code':

The following products depend on Canada for nickel:
*   **FieldSprayer_BE9WW (35298723)** depends on PrecisionBolt_66KS7 (E0301006) which sources nickel for alloy from Canada.
*   **SeedPlanter_DJG7M (35298721)** depends on PrecisionBolt_66KS7 (E0301006) which sources nickel for alloy from Canada.
*   **FarmTractor_2DYUV (35701269)** depends on PrecisionBolt_252A1 (D0301002) through CabWiringUnit_EVKCK (13978746), which sources nickel from Canada.
*   **SeedPlanter_WM8Z2 (35561992)** depends on PrecisionBolt_252A1 (D0301002) through CabWiringUnit_EVKCK (13978746), which sources nickel from Canada.
*   **CropHarvester_ZHSAB (35733612)** depends on PrecisionBolt_252A1 (D0301002) through CabWiringUnit_EVKCK (13978746), which sources nickel from Canada.
*   **FarmTractor_VLM4F (35513559)** depends on PrecisionBolt_252A1 (D0301002) through CabWiringUnit_EVKCK (13978746), which sources nickel from Canada.
*   **FarmTractor_2QT51 (35733576)** depends on PrecisionBolt_252A1 (D0301002) through CabWiringUnit_EVKCK (13978746), which sources nickel from Canada.
*   **CropHarvester_V1GUY (35861688)** depends on PrecisionBolt_252A1 (D0301002) through ChassisFrame_3AC3Z (13952372), which sources nickel from Canada.
*   **HayCollector_4APNS (35775932)** depends on PrecisionBolt_252A1 (D0301002) through ChassisFrame_3AC3Z (13952372), which sources nickel from Canada.
*   **HayCollector_UWIBH (35839618)** depends on PrecisionBolt_252A1 (D0301002) through ChassisFrame_3AC3Z (13952372), which sources nickel from Canada.
*   **FieldSprayer_KHWK2 (35764312)** depends on PrecisionBolt_252A1 (D0301002) through ChassisFrame_3AC3Z (13952372), which sources nickel from Canada.
*   **HayCollector_6S90S (35764334)** depends on PrecisionBolt_252A1 (D0301002) through ChassisFrame_3AC3Z (13952372), which sources nickel from Canada.
*   **CropHarvester_ABK76 (35860398)** depends on PrecisionBolt_252A1 (D0301002) through ChassisFrame_3AC3Z (13952372), which sources nickel from Canada.
*   **SeedPlanter_3J2JW (35839611)** depends on PrecisionBolt_252A1 (D0301002) through ChassisFrame_3AC3Z (13952372), which sources nickel from Canada.
*   **FarmTractor_VOL6Y (35775931)** depends on PrecisionBolt_252A1 (D0301002) through ChassisFrame_3AC3Z (13952372), which sources nickel from Canada.
*   **FarmTractor_IUP4U (35845787)** depends on PrecisionBolt_252A1 (D0301002) through ChassisFrame_3AC3Z (13952372), which sources nickel from Canada.
*   **FieldSprayer_63AJB (35839613)** depends on PrecisionBolt_252A1 (D0301002) through ChassisFrame_3AC3Z (13952372), which sources nickel from Canada.
*   **FarmTractor_HKVNA (35841324)** depends on PrecisionBolt_252A1 (D0301002) through ChassisFrame_3AC3Z (13952372), which sources nickel from Canada.
*   **FarmTractor_LA44E (35764331)** depends on PrecisionBolt_252A1 (D0301002) through ChassisFrame_3AC3Z (13952372), which sources nickel from Canada.
*   **SeedPlanter_1P2R4 (35841333)** depends on PrecisionBolt_252A1 (D0301002) through ChassisFrame_3AC3Z (13952372), which sources nickel from Canada.
*   **CropHarvester_YSN76 (35839620)** depends on PrecisionBolt_252A1 (D0301002) through ChassisFrame_3AC3Z (13952372), which sources nickel from Canada.
*   **SeedPlanter_D9K9H (35861691)** depends on PrecisionBolt_252A1 (D0301002) through ChassisFrame_3AC3Z (13952372), which sources nickel from Canada.
*   **HayCollector_RVK3E (35866296)** depends on PrecisionBolt_252A1 (D0301002) through ChassisFrame_3AC3Z (13952372), which sources nickel from Canada.
*   **CropHarvester_9YZB9 (35764327)** depends on PrecisionBolt_252A1 (D0301002) through ChassisFrame_3AC3Z (13952372), which sources nickel from Canada.

In [58]:
res = await run_prompt('Supplier YP8UYT had a fire and is out of ooperation. I need to source alternatives.  Can you help?')
print("\n\n\n\nFinal Response:")
display(Markdown(res))

None id='adk-bcf84548-7a1c-4ab7-8e60-fdb9b4f2ff6a' args={'supplier_code': 'YP8UYT'} name='get_supplier_substitutions' None
None None will_continue=None scheduling=None id='adk-bcf84548-7a1c-4ab7-8e60-fdb9b4f2ff6a' name='get_supplier_substitutions' response={'result': [{'sku_id': '28710197', 'name': 'Harness_YU5KA', 'recommended_supplier': {'distance': 100.0, 'code': 'T4J5DE'}, 'other_suppliers': []}, {'sku_id': '28018274', 'name': 'Clamp_MRVR0', 'recommended_supplier': {'distance': 100.0, 'code': 'T4J5DE'}, 'other_suppliers': [{'distance': 1377.0, 'code': 'RO6UY3'}, {'distance': 11170.0, 'code': '6TIPXK'}]}, {'sku_id': 'DED9021006200', 'name': 'HydraulicSystem_KY7NM', 'recommended_supplier': {'distance': 100.0, 'code': 'T4J5DE'}, 'other_suppliers': [{'distance': 11218.0, 'code': '67F8ZB'}]}, {'sku_id': '28189847', 'name': 'Sensor_M5TC1', 'recommended_supplier': {'distance': 100.0, 'code': 'T4J5DE'}, 'other_suppliers': [{'distance': 134.0, 'code': 'EY2DU5'}, {'distance': 1359.0, 'code':

I found the following substitution recommendations for items supplied by YP8UYT:

*   **Harness_YU5KA (28710197)**:
    *   Recommended: T4J5DE (100 miles away)
*   **Clamp_MRVR0 (28018274)**:
    *   Recommended: T4J5DE (100 miles away)
    *   Others: RO6UY3 (1377 miles), 6TIPXK (11170 miles)
*   **HydraulicSystem_KY7NM (DED9021006200)**:
    *   Recommended: T4J5DE (100 miles away)
    *   Others: 67F8ZB (11218 miles)
*   **Sensor_M5TC1 (28189847)**:
    *   Recommended: T4J5DE (100 miles away)
    *   Others: EY2DU5 (134 miles), CC2XO0 (1359 miles), RO6UY3 (1377 miles), 6TIPXK (11170 miles), ZCOX6M (11218 miles)
*   **Pulley_JG25U (28428194)**:
    *   Recommended: T4J5DE (100 miles away)
*   **Gear_HIE2T (16228496)**:
    *   Recommended: T4J5DE (100 miles away)
    *   Others: JYRKG2 (100 miles), EY2DU5 (134 miles), TCUA8H (228 miles), RO6UY3 (1377 miles), PQSIH9 (1436 miles), Z09DJG (1627 miles), OY1UE5 (10187 miles), R0AZ6C (10415 miles), NLBFEK (10642 miles), 6TIPXK (11170 miles), 67F8ZB (11218 miles), LDZSV0 (11218 miles), 3VOH4R (11218 miles), ABGQV8 (11218 miles), DSKDW4 (11218 miles)
*   **Belt_TC5XP (28443180)**:
    *   Recommended: T4J5DE (100 miles away)
    *   Others: RO6UY3 (1377 miles)
*   **Bearing_XMY98 (28469393)**:
    *   Recommended: WOYEWL (0 miles away)
    *   Others: T4J5DE (100 miles), JYRKG2 (100 miles), 0MW25Z (208 miles), FP0QV8 (208 miles), OY1UE5 (10187 miles), 67F8ZB (11218 miles), ABGQV8 (11218 miles)
*   **GearSet_7MCLV (16249305)**:
    *   Recommended: T4J5DE (100 miles away)
    *   Others: JYRKG2 (100 miles), F9MVG2 (432 miles), RO6UY3 (1377 miles), PQSIH9 (1436 miles), 1AS9EK (1627 miles), OY1UE5 (10187 miles), 6TIPXK (11170 miles), ABGQV8 (11218 miles), ZCOX6M (11218 miles), DSKDW4 (11218 miles)
*   **HydraulicFitting_B24BJ (28202339)**:
    *   Recommended: T4J5DE (100 miles away)
    *   Others: JYRKG2 (100 miles), F9MVG2 (432 miles), MX9142 (611 miles), OKNNDD (1220 miles), RO6UY3 (1377 miles), NLBFEK (10642 miles), 6TIPXK (11170 miles), 67F8ZB (11218 miles), LDZSV0 (11218 miles), ABGQV8 (11218 miles), ZCOX6M (11218 miles)
*   **Clamp_P9GLY (28232078)**:
    *   Recommended: T4J5DE (100 miles away)
    *   Others: JYRKG2 (100 miles), F9MVG2 (432 miles), RO6UY3 (1377 miles), Z09DJG (1627 miles), THRFXZ (10187 miles), NLBFEK (10642 miles), 6TIPXK (11170 miles), 67F8ZB (11218 miles), LDZSV0 (11218 miles), ABGQV8 (11218 miles)
*   **GearSet_QVLA1 (28083943)**:
    *   Recommended: T4J5DE (100 miles away)
    *   Others: 36G09B (332 miles), AYFNW0 (332 miles), GZWGBJ (332 miles), RO6UY3 (1377 miles), OY1UE5 (10187 miles), NLBFEK (10642 miles), 6YIZBB (11161 miles), 6TIPXK (11170 miles)
*   **CabWiringUnit_CA92P (DED9230001530)**:
    *   Recommended: T4J5DE (100 miles away)

I hope this helps you find alternative sources!

In [61]:
session_list = await runner.session_service.list_sessions(app_name=APP_NAME, user_id=USER_ID)
for session in session_list.sessions:
  print(session.model_dump())

{'id': 'cf790855-7dda-407c-a425-649192a33cda', 'app_name': 'BOM & Supplier Analyst', 'user_id': 'Zach Blumenfeld', 'state': {}, 'events': [], 'last_update_time': 1748523160.734146}


In [62]:
result = await run_prompt("Summarize the results of the previous research questions")
print("\n\n\n\nFinal Response:")
display(Markdown(result))

Okay, I can summarize the results of our previous research:

1.  **Finished Product Types:** We identified several families of finished products, including FarmTractor (21,521 products), SeedPlanter (21,488 products), FieldSprayer (21,519 products), CropHarvester (21,406 products), HayCollector (21,671 products), and smaller categories like CabWiringUnit, ControlAssembly, MachineRig, HydraulicSystem, ChassisFrame, and DrivePlatform.

2.  **BOM and Country Dependencies for Product 35472245A:**
    *   **BOM:** Product 35472245A (CabWiringUnit_Z5OO3) has a complex Bill of Materials, depending directly on numerous components like M6495003X (DrivePlatform_SBK8U) and indirectly on others such as M7307U91X (RawWire_YCBEO) through intermediate components.
    *   **Country Dependencies:** This product has several potential country dependencies stemming from its components:
        *   **M7307U91X (RawWire_YCBEO):** Sourcing raw copper cathode from Chile and Peru; drawing and annealing facilit

Okay, I can summarize the results of our previous research:

1.  **Finished Product Types:** We identified several families of finished products, including FarmTractor (21,521 products), SeedPlanter (21,488 products), FieldSprayer (21,519 products), CropHarvester (21,406 products), HayCollector (21,671 products), and smaller categories like CabWiringUnit, ControlAssembly, MachineRig, HydraulicSystem, ChassisFrame, and DrivePlatform.

2.  **BOM and Country Dependencies for Product 35472245A:**
    *   **BOM:** Product 35472245A (CabWiringUnit_Z5OO3) has a complex Bill of Materials, depending directly on numerous components like M6495003X (DrivePlatform_SBK8U) and indirectly on others such as M7307U91X (RawWire_YCBEO) through intermediate components.
    *   **Country Dependencies:** This product has several potential country dependencies stemming from its components:
        *   **M7307U91X (RawWire_YCBEO):** Sourcing raw copper cathode from Chile and Peru; drawing and annealing facilities in Mexico and Viet Nam.
        *   **M7307U92X (PrecisionBolt_O12J1):** Final machining in the USA; sourcing raw titanium sponge from Japan; procuring alloying elements from the Russian Federation.
        *   **M7307U93X (Fastener_CP82S):** Precision forging in Germany; sourcing raw titanium from the Russian Federation.

3.  **Products Depending on Canada for Nickel:** We found a list of finished products that rely on components sourcing nickel from Canada. These components include PrecisionBolt_66KS7 (E0301006) and PrecisionBolt_252A1 (D0301002). Affected product families include various models of FieldSprayers, SeedPlanters, FarmTractors, CropHarvesters, and HayCollectors. For instance, FieldSprayer_BE9WW (35298723) and FarmTractor_2DYUV (35701269) are among those listed.

4.  **Alternative Sourcing for Supplier YP8UYT:** Due to a fire at supplier YP8UYT, we identified alternative suppliers for the items they provide. For many items like Harness_YU5KA (28710197), Clamp_MRVR0 (28018274), and HydraulicSystem_KY7NM (DED9021006200), supplier T4J5DE (located 100 miles away) was recommended. For some items, like Bearing_XMY98 (28469393), a closer alternative WOYEWL (0 miles away) was found. Other more distant alternatives were also listed for various components.

In [64]:
session_list = await runner.session_service.list_sessions(app_name=APP_NAME, user_id=USER_ID)
for session in session_list.sessions:
  print(f"Deleting session {session}")
  await runner.session_service.delete_session(app_name=APP_NAME, user_id=USER_ID, session_id=session.id)

Deleting session id='cf790855-7dda-407c-a425-649192a33cda' app_name='BOM & Supplier Analyst' user_id='Zach Blumenfeld' state={} events=[] last_update_time=1748523249.560376
